In [ ]:
!git clone https://github.com/Ibrahimrahhal/nudity-dataset.git

Cloning into 'nudity-dataset'...
remote: Enumerating objects: 4370, done.
remote: Counting objects: 100% (14/14), done.
remote: Compressing objects: 100% (12/12), done.
remote: Total 4370 (delta 4), reused 9 (delta 2), pack-reused 4356
Receiving objects: 100% (4370/4370), 844.63 MiB | 37.01 MiB/s, done.
Resolving deltas: 100% (4/4), done.
Updating files: 100% (4211/4211), done.


In [ ]:
!rm -rf nudity-dataset/filter.py

In [ ]:
!pip install tensorflow numpy matplotlib


In [ ]:
import os
import shutil

print(os.listdir("nudity-dataset/"))
shutil.rmtree("nudity-dataset/.git")
print(os.listdir("nudity-dataset/"))


['neither-nor', 'nudity-scenes']


FileNotFoundError: [Errno 2] No such file or directory: 'nudity-dataset/.git'

In [ ]:
import numpy as np
import os
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications.resnet50 import preprocess_input

# Enhanced augmentation
train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=40,  # randomly rotate images in the range (degrees, 0 to 180)
    width_shift_range=0.2,  # randomly shift images horizontally (fraction of total width)
    height_shift_range=0.2,  # randomly shift images vertically (fraction of total height)
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,  # randomly flip images
    fill_mode='nearest',  # strategy used for filling in newly created pixels
    validation_split=0.2,  # using 20% of the data for validation
    preprocessing_function=preprocess_input)

train_generator = train_datagen.flow_from_directory(
    'nudity-dataset/',  # this is the target directory
    target_size=(224, 224),  # resize images to 224x224 for ResNet50
    batch_size=3,
    class_mode='binary',  # binary labels for nude and non-nude
    subset='training')  # set as training data

validation_generator = train_datagen.flow_from_directory(
    'nudity-dataset/',  # same directory as training data
    target_size=(224, 224),
    batch_size=3,
    class_mode='binary',
    subset='validation')  # set as validation data


Found 2580 images belonging to 2 classes.
Found 644 images belonging to 2 classes.


In [ ]:
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D, Dropout
from keras import models, regularizers, layers, optimizers, losses, metrics
from tensorflow.keras.optimizers import Adam


import tensorflow as tf
base_model = ResNet50(weights='imagenet', include_top=False)

# Create the model
x = base_model.output
x = GlobalAveragePooling2D()(x)  # Add GAP layer
x = Dense(256, activation='relu')(x)  # Add a fully connected layer
x = Dropout(0.5)(x)  # Add a dropout layer for regularization
predictions = Dense(1, activation='sigmoid')(x)  # Output layer for binary classification

# This creates a model that includes the base model and two additional layers
model = Model(inputs=base_model.input, outputs=predictions)

# Compile the model
model.compile(optimizer=Adam(lr=0.0001), loss='binary_crossentropy', metrics=['accuracy'])



for layer in base_model.layers[:165]:
   layer.trainable = False
for layer in base_model.layers[165:]:
   layer.trainable = True

model.compile(optimizer=optimizers.Adam(),
              loss='binary_crossentropy',
              metrics=['accuracy'])

94765736/94765736 [==============================] - 1s 0us/step


In [ ]:
for i, layer in enumerate(model.layers):
   print(i, layer.name, layer.trainable)


0 input_13 False
1 conv1_pad False
2 conv1_conv False
3 conv1_bn False
4 conv1_relu False
5 pool1_pad False
6 pool1_pool False
7 conv2_block1_1_conv False
8 conv2_block1_1_bn False
9 conv2_block1_1_relu False
10 conv2_block1_2_conv False
11 conv2_block1_2_bn False
12 conv2_block1_2_relu False
13 conv2_block1_0_conv False
14 conv2_block1_3_conv False
15 conv2_block1_0_bn False
16 conv2_block1_3_bn False
17 conv2_block1_add False
18 conv2_block1_out False
19 conv2_block2_1_conv False
20 conv2_block2_1_bn False
21 conv2_block2_1_relu False
22 conv2_block2_2_conv False
23 conv2_block2_2_bn False
24 conv2_block2_2_relu False
25 conv2_block2_3_conv False
26 conv2_block2_3_bn False
27 conv2_block2_add False
28 conv2_block2_out False
29 conv2_block3_1_conv False
30 conv2_block3_1_bn False
31 conv2_block3_1_relu False
32 conv2_block3_2_conv False
33 conv2_block3_2_bn False
34 conv2_block3_2_relu False
35 conv2_block3_3_conv False
36 conv2_block3_3_bn False
37 conv2_block3_add False
38 conv2_blo

In [ ]:
history = model.fit(
    train_generator,
    steps_per_epoch=train_generator.samples// train_generator.batch_size,
    epochs=40,
    validation_data=validation_generator,
    validation_steps=validation_generator.samples // validation_generator.batch_size)


Epoch 1/30
860/860 [==============================] - 80s 92ms/step - loss: 0.6655 - accuracy: 0.6178 - val_loss: 0.8268 - val_accuracy: 0.6184
Epoch 2/30
860/860 [==============================] - 80s 92ms/step - loss: 0.6646 - accuracy: 0.6202 - val_loss: 0.6551 - val_accuracy: 0.6199
Epoch 3/30
860/860 [==============================] - 79s 92ms/step - loss: 0.6646 - accuracy: 0.6167 - val_loss: 0.6683 - val_accuracy: 0.6199
Epoch 4/30
860/860 [==============================] - 79s 92ms/step - loss: 0.6622 - accuracy: 0.6233 - val_loss: 0.6593 - val_accuracy: 0.6168
Epoch 5/30
860/860 [==============================] - 80s 93ms/step - loss: 0.6611 - accuracy: 0.6233 - val_loss: 0.6690 - val_accuracy: 0.6215
Epoch 6/30
860/860 [==============================] - 79s 92ms/step - loss: 0.6620 - accuracy: 0.6182 - val_loss: 0.6729 - val_accuracy: 0.6121
Epoch 7/30
860/860 [==============================] - 78s 91ms/step - loss: 0.6601 - accuracy: 0.6229 - val_loss: 0.6801 - val_accuracy:

In [ ]:
# Evaluate the model on the validation set
val_loss, val_accuracy = model.evaluate(validation_generator)
print(f"Validation loss: {val_loss}")
print(f"Validation accuracy: {val_accuracy}")


21/21 [==============================] - 12s 579ms/step - loss: 0.1518 - accuracy: 0.7644
Validation loss: 0.1518694281578064
Validation accuracy: 0.7644099247455597


In [ ]:
model.save('model')

In [ ]:
print(model.summary())

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_1 (InputLayer)        [(None, None, None, 3)]      0         []                            
                                                                                                  
 conv1_pad (ZeroPadding2D)   (None, None, None, 3)        0         ['input_1[0][0]']             
                                                                                                  
 conv1_conv (Conv2D)         (None, None, None, 64)       9472      ['conv1_pad[0][0]']           
                                                                                                  
 conv1_bn (BatchNormalizati  (None, None, None, 64)       256       ['conv1_conv[0][0]']          
 on)                                                                                          